In [1]:
import tensorflow as tf
import os
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from inspect import signature
from PIL import Image
from scipy.cluster.vq import kmeans
import matplotlib.pyplot as plt
from multiprocessing import Pool
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ReduceLROnPlateau
from decimal import *



# folder where all *.npy files are located
path_to_data = r'/home/saban/Project/62_classes/'

# create log files in `current dir/`
logs = 'logs/' 

reference_features = np.load(os.path.join(path_to_data, 'training_features_62_classes.npy'))
reference_features = reference_features / np.linalg.norm(reference_features, 2, axis=1, keepdims=True)
reference_classes = np.load(os.path.join(path_to_data, 'training_classids_62_classes.npy'))
reference_probs = np.load(os.path.join(path_to_data, 'training_probabilities_62_classes.npy'))

incremental_features = np.load(os.path.join(path_to_data, 'validation_features_62_classes.npy'))

incremental_features = incremental_features / np.linalg.norm(incremental_features, 2 , axis=1, keepdims=True)
incremental_classes = np.load(os.path.join(path_to_data, 'validation_classids_62_classes.npy'))
incremental_probs = np.load(os.path.join(path_to_data, 'validation_probabilities_62_classes.npy'))

test_features = np.load(os.path.join(path_to_data, 'test_features_62_classes.npy'))
test_features = test_features / np.linalg.norm(test_features, 2 , axis=1, keepdims=True)
test_classes = np.load(os.path.join(path_to_data, 'test_classids_62_classes.npy'))
test_probs = np.load(os.path.join(path_to_data, 'test_probabilities_62_classes.npy'))

class_mapping = np.load(os.path.join(path_to_data, 'class_mapping_62_classes.npy'), allow_pickle=True).item()
unique_classes = np.unique(reference_classes)

test_dataset = tf.data.Dataset.from_tensor_slices((test_features, tf.keras.utils.to_categorical(test_classes)))
test_dataset = test_dataset.batch(32)


In [ ]:
print(reference_features)

In [2]:
def soft_distance(x, features, gamma):
    diff = x - features          
    dist = np.sqrt(np.square(diff)).sum(axis=-1)
    return np.exp(-gamma * dist).sum()

def calculate_distances(features, anchor, gamma):
    labels = np.zeros((features.shape[0],))
    probs = np.zeros_like(labels)
    for i, feature in enumerate(features): 
        distances = np.zeros((len(unique_classes),))
        for cid in unique_classes:            
            n = soft_distance(feature, anchor[cid], gamma)            
            ds = np.finfo(float).eps
            for ccid in unique_classes:                
                d = soft_distance(feature, anchor[ccid], gamma)
                ds += d
            
            distances[cid] = n/ds
            
        labels[i] = distances.argmax()
        probs[i]  = distances.max()
    return labels, probs

def get_soft_voting(features, anchor, gamma=5.0):       
    labels = []
    probs = []
    cpus = os.cpu_count() - 2 # keep two cpus please
    num_chunks = features.shape[0] // (cpus - 1)
    
    with Pool(processes=cpus) as pool:
        results = []

        for chunk in range(cpus):
            start = chunk * num_chunks
            
            if chunk == cpus - 1:
                sub_features = features[start:]
            else:
                sub_features = features[start:start + num_chunks]
            
            results.append(pool.apply_async(calculate_distances, args=(sub_features, anchor, gamma)))
        
        with tqdm_notebook(total=features.shape[0]) as pbar:        
            for result in results:
                label, prob = result.get()
                labels.extend(label)
                probs.extend(prob)
                pbar.update(len(label))
        
    return labels, probs

def get_anchor_points(num_points):
    anchor_points = []
    for cid in tqdm_notebook(unique_classes, desc='calculate anchor points'):
        indices = np.where(reference_classes == cid)[0]
        class_features = reference_features[indices]
        codebook, _ = kmeans(class_features, num_points, iter=20)
        #codebook /= np.linalg.norm(codebook, 2, axis=1, keepdims=True)
        
        anchor_points += [codebook]
    return np.array(anchor_points)

def calculate_weights(features, name):
    if name == 'baseline':
        return np.ones(features.shape[0])
    if name == 'weighted_prob':
        weight_prob = np.zeros_like(probs)
        for pr in range(len(probs)):
            weight_prob[pr] = np.float(1.0) - (probs[pr])
        return np.array(weight_prob)
    if name == 'active_0.1':
        prob_test=(1.0-np.array(probs))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.1:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_0.3':
        prob_test=(1.0-np.array(probs))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.3:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_0.5':
        prob_test=(1.0-np.array(probs))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.5:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_0.7':
        prob_test=(1.0-np.array(probs))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if prob_test[i] >= 0.7:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)
    if name == 'active_middle':
        prob_test=(1.0-np.array(probs))
        prob_test_new = []
        index = []
        for i in range(len(prob_test)):
            if 0.5 >= prob_test[i] >= 0.1:
                prob_test_new.append((prob_test[i]))
                index.append(i)
        return np.array(prob_test_new), np.array(index)

In [3]:
anchors_new = get_anchor_points(10)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:54: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [4]:
anchors = anchors_new

In [5]:
np.save('anchors_not_norm',)

In [3]:
anchors = np.load('anchors_not_norm.npy')

In [7]:
anchors.shape


(62, 10, 2048)

In [5]:
labels_soft_5, probs_new = get_soft_voting(incremental_features, anchors, gamma=1.5)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:43: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [6]:
labels_soft = labels_soft_5
probs = probs_new

In [7]:
np.save('labels_increment_soft_voting',)
np.save('probs_increment_soft_voting',)

In [4]:
labels_soft = np.load('labels_increment_soft_voting.npy')
probs = np.load('probs_increment_soft_voting.npy')

In [6]:
prob_test=(1.0-np.array(probs))
prob_test_new = []
index = []
for i in range(len(prob_test)):
    if prob_test[i] >= 0.9:
        prob_test_new.append((prob_test[i]))
        index.append(i)
        

In [7]:
print(index)
print(len(prob_test_new))

[1584, 2069, 5767, 5803, 5837, 6742, 7574, 7738, 8961, 13501, 14086, 15544]
12


In [8]:
len(incremental_classes[index])

12

In [9]:
accu = accuracy_score(incremental_classes, labels_soft )
print(accu)

0.7861617148439745


In [10]:
accu_softmax = accuracy_score(incremental_classes, incremental_probs.argmax(-1) )
print(accu_softmax)

0.6671455663467617


In [24]:
experiment_weights_types = ['baseline']
for name in experiment_weights_types:
    for attempt in range(2):
        # sample weights 
        weights = calculate_weights(incremental_features, name)

        incremental_dataset = tf.data.Dataset.from_tensor_slices((incremental_features, tf.keras.utils.to_categorical(labels_soft),weights))
        incremental_dataset = incremental_dataset.shuffle(2048).batch(32)
        # load pre-trained keras model
        model = tf.keras.models.load_model(os.path.join(path_to_data, 'submodel_62_classes.hdf5'), compile=False)

        # use SGD optimizer
        optimizer = optimizers.Adam(lr=.0001)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # define callbacks 
        callbacks = [
            ModelCheckpoint(
                os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)), monitor='val_accuracy', 
                verbose=0, save_best_only=True, save_weights_only=False, mode='auto'),
            # you can use tensorboard --logdir path/to/log/dir to visualize the training parameters like loss and accuracy
            TensorBoard(os.path.join(logs, name, str(attempt))),
            #ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,patience=10, min_lr=0.000001, verbose=1)

        ] 

        # start training for 250 epochs
        model.fit_generator(
            incremental_dataset,
            epochs=500,
            callbacks=callbacks,
            validation_data=test_dataset)

        model.load_weights(os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)))
        
        y_pred = model.predict(test_dataset)
        cm_b = confusion_matrix(test_classes, np.array(y_pred).argmax(axis=1))
        
        np.save(os.path.join('cm_probablity_baseline_'+str(attempt)),cm_b)
        print(os.path.join('cm_probablity_baseline_'+str(attempt)),y_pred)


Train for 544 steps, validate for 969 steps
Epoch 1/500
544/544 [==============================] - 4s 7ms/step - loss: 3.8281 - accuracy: 0.5535 - val_loss: 3.7354 - val_accuracy: 0.6074
Epoch 2/500
544/544 [==============================] - 3s 6ms/step - loss: 3.6216 - accuracy: 0.6652 - val_loss: 3.5582 - val_accuracy: 0.6025
Epoch 3/500
544/544 [==============================] - 4s 7ms/step - loss: 3.4334 - accuracy: 0.7036 - val_loss: 3.3892 - val_accuracy: 0.6093
Epoch 4/500
544/544 [==============================] - 3s 6ms/step - loss: 3.2546 - accuracy: 0.7261 - val_loss: 3.2282 - val_accuracy: 0.6205
Epoch 5/500
544/544 [==============================] - 3s 6ms/step - loss: 3.0851 - accuracy: 0.7415 - val_loss: 3.0751 - val_accuracy: 0.6326
Epoch 6/500
544/544 [==============================] - 4s 7ms/step - loss: 2.9240 - accuracy: 0.7589 - val_loss: 2.9295 - val_accuracy: 0.6411
Epoch 7/500
544/544 [==============================] - 4s 7ms/step - loss: 2.7709 - accuracy: 0.76

Epoch 114/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2975 - accuracy: 0.9476 - val_loss: 0.7158 - val_accuracy: 0.7885
Epoch 115/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2955 - accuracy: 0.9486 - val_loss: 0.7152 - val_accuracy: 0.7886
Epoch 116/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2935 - accuracy: 0.9491 - val_loss: 0.7145 - val_accuracy: 0.7887
Epoch 117/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2915 - accuracy: 0.9489 - val_loss: 0.7140 - val_accuracy: 0.7888
Epoch 118/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2895 - accuracy: 0.9496 - val_loss: 0.7135 - val_accuracy: 0.7889
Epoch 119/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2876 - accuracy: 0.9496 - val_loss: 0.7128 - val_accuracy: 0.7890
Epoch 120/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2858 - accuracy: 0.9502 - val_loss: 0.7123 - val_ac

544/544 [==============================] - 4s 7ms/step - loss: 0.1672 - accuracy: 0.9749 - val_loss: 0.7076 - val_accuracy: 0.7958
Epoch 227/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1664 - accuracy: 0.9756 - val_loss: 0.7078 - val_accuracy: 0.7961
Epoch 228/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1658 - accuracy: 0.9758 - val_loss: 0.7080 - val_accuracy: 0.7960
Epoch 229/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1651 - accuracy: 0.9758 - val_loss: 0.7081 - val_accuracy: 0.7959
Epoch 230/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1644 - accuracy: 0.9757 - val_loss: 0.7085 - val_accuracy: 0.7960
Epoch 231/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1637 - accuracy: 0.9762 - val_loss: 0.7085 - val_accuracy: 0.7960
Epoch 232/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1630 - accuracy: 0.9763 - val_loss: 0.7088 - val_accuracy: 0.7960

544/544 [==============================] - 4s 7ms/step - loss: 0.1085 - accuracy: 0.9890 - val_loss: 0.7364 - val_accuracy: 0.7975
Epoch 339/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1081 - accuracy: 0.9890 - val_loss: 0.7368 - val_accuracy: 0.7975
Epoch 340/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1078 - accuracy: 0.9893 - val_loss: 0.7371 - val_accuracy: 0.7975
Epoch 341/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1074 - accuracy: 0.9891 - val_loss: 0.7374 - val_accuracy: 0.7975
Epoch 342/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1070 - accuracy: 0.9893 - val_loss: 0.7377 - val_accuracy: 0.7975
Epoch 343/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1066 - accuracy: 0.9894 - val_loss: 0.7381 - val_accuracy: 0.7975
Epoch 344/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1062 - accuracy: 0.9895 - val_loss: 0.7385 - val_accuracy: 0.7975

544/544 [==============================] - 4s 7ms/step - loss: 0.0738 - accuracy: 0.9953 - val_loss: 0.7736 - val_accuracy: 0.7973
Epoch 451/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0735 - accuracy: 0.9952 - val_loss: 0.7740 - val_accuracy: 0.7972
Epoch 452/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0733 - accuracy: 0.9954 - val_loss: 0.7743 - val_accuracy: 0.7973
Epoch 453/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0731 - accuracy: 0.9954 - val_loss: 0.7746 - val_accuracy: 0.7973
Epoch 454/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0728 - accuracy: 0.9954 - val_loss: 0.7749 - val_accuracy: 0.7972
Epoch 455/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0726 - accuracy: 0.9955 - val_loss: 0.7753 - val_accuracy: 0.7972
Epoch 456/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0724 - accuracy: 0.9956 - val_loss: 0.7756 - val_accuracy: 0.7973

544/544 [==============================] - 4s 7ms/step - loss: 0.5070 - accuracy: 0.9210 - val_loss: 0.8285 - val_accuracy: 0.7752
Epoch 60/500
544/544 [==============================] - 4s 7ms/step - loss: 0.4998 - accuracy: 0.9221 - val_loss: 0.8236 - val_accuracy: 0.7754
Epoch 61/500
544/544 [==============================] - 4s 7ms/step - loss: 0.4928 - accuracy: 0.9232 - val_loss: 0.8187 - val_accuracy: 0.7758
Epoch 62/500
544/544 [==============================] - 4s 7ms/step - loss: 0.4860 - accuracy: 0.9238 - val_loss: 0.8143 - val_accuracy: 0.7760
Epoch 63/500
544/544 [==============================] - 4s 7ms/step - loss: 0.4792 - accuracy: 0.9237 - val_loss: 0.8098 - val_accuracy: 0.7768
Epoch 64/500
544/544 [==============================] - 4s 7ms/step - loss: 0.4729 - accuracy: 0.9249 - val_loss: 0.8057 - val_accuracy: 0.7770
Epoch 65/500
544/544 [==============================] - 4s 7ms/step - loss: 0.4667 - accuracy: 0.9258 - val_loss: 0.8018 - val_accuracy: 0.7776
Epoch

Epoch 116/500
544/544 [==============================] - 4s 6ms/step - loss: 0.2935 - accuracy: 0.9487 - val_loss: 0.7144 - val_accuracy: 0.7888
Epoch 117/500
544/544 [==============================] - 3s 6ms/step - loss: 0.2914 - accuracy: 0.9487 - val_loss: 0.7138 - val_accuracy: 0.7889
Epoch 118/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2895 - accuracy: 0.9491 - val_loss: 0.7132 - val_accuracy: 0.7890
Epoch 119/500
544/544 [==============================] - 3s 6ms/step - loss: 0.2876 - accuracy: 0.9496 - val_loss: 0.7128 - val_accuracy: 0.7890
Epoch 120/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2857 - accuracy: 0.9501 - val_loss: 0.7122 - val_accuracy: 0.7892
Epoch 121/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2838 - accuracy: 0.9505 - val_loss: 0.7117 - val_accuracy: 0.7894
Epoch 122/500
544/544 [==============================] - 4s 7ms/step - loss: 0.2820 - accuracy: 0.9506 - val_loss: 0.7112 - val_ac

544/544 [==============================] - 4s 7ms/step - loss: 0.1658 - accuracy: 0.9756 - val_loss: 0.7080 - val_accuracy: 0.7959
Epoch 229/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1650 - accuracy: 0.9758 - val_loss: 0.7082 - val_accuracy: 0.7959
Epoch 230/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1644 - accuracy: 0.9760 - val_loss: 0.7083 - val_accuracy: 0.7959
Epoch 231/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1637 - accuracy: 0.9762 - val_loss: 0.7085 - val_accuracy: 0.7961
Epoch 232/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1630 - accuracy: 0.9764 - val_loss: 0.7087 - val_accuracy: 0.7959
Epoch 233/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1623 - accuracy: 0.9765 - val_loss: 0.7089 - val_accuracy: 0.7960
Epoch 234/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1616 - accuracy: 0.9763 - val_loss: 0.7092 - val_accuracy: 0.7960

544/544 [==============================] - 4s 7ms/step - loss: 0.1077 - accuracy: 0.9890 - val_loss: 0.7371 - val_accuracy: 0.7976
Epoch 341/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1074 - accuracy: 0.9891 - val_loss: 0.7375 - val_accuracy: 0.7976
Epoch 342/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1070 - accuracy: 0.9891 - val_loss: 0.7378 - val_accuracy: 0.7974
Epoch 343/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1066 - accuracy: 0.9891 - val_loss: 0.7382 - val_accuracy: 0.7975
Epoch 344/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1062 - accuracy: 0.9894 - val_loss: 0.7384 - val_accuracy: 0.7975
Epoch 345/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1058 - accuracy: 0.9894 - val_loss: 0.7387 - val_accuracy: 0.7975
Epoch 346/500
544/544 [==============================] - 4s 7ms/step - loss: 0.1054 - accuracy: 0.9897 - val_loss: 0.7390 - val_accuracy: 0.7975

544/544 [==============================] - 4s 7ms/step - loss: 0.0733 - accuracy: 0.9954 - val_loss: 0.7743 - val_accuracy: 0.7972
Epoch 453/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0730 - accuracy: 0.9953 - val_loss: 0.7746 - val_accuracy: 0.7973
Epoch 454/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0728 - accuracy: 0.9955 - val_loss: 0.7748 - val_accuracy: 0.7971
Epoch 455/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0726 - accuracy: 0.9955 - val_loss: 0.7753 - val_accuracy: 0.7973
Epoch 456/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0723 - accuracy: 0.9955 - val_loss: 0.7757 - val_accuracy: 0.7972
Epoch 457/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0721 - accuracy: 0.9955 - val_loss: 0.7760 - val_accuracy: 0.7972
Epoch 458/500
544/544 [==============================] - 4s 7ms/step - loss: 0.0718 - accuracy: 0.9956 - val_loss: 0.7763 - val_accuracy: 0.7972

In [ ]:
np.ones(incremental_features.shape[0]).shape

In [20]:
y_pred = model.predict(test_dataset)
    
cm_b = confusion_matrix(test_classes, np.array(y_pred).argmax(axis=1))

In [21]:
print(np.array(y_pred).argmax(axis=1))

[ 0  0 26 ... 61 61  7]


In [ ]:
print(test_classes)

In [ ]:
print(cm)

In [22]:
pred_accu = accuracy_score(test_classes, np.array(y_pred).argmax(axis=1) )
print(pred_accu)

0.7978387096774193


In [23]:
np.save(os.path.join('cm_probablity_baseline_'+str(attempt)),cm_b)

In [68]:
np.save('cm_probablity_baseline_1',cm_b)

In [12]:
experiment_weights_types = ['active_middle']
for name in experiment_weights_types:
    for attempt in range(2):
        # sample weights 
        weights,ind = calculate_weights(incremental_features, name)

        incremental_dataset = tf.data.Dataset.from_tensor_slices((incremental_features[ind], tf.keras.utils.to_categorical(incremental_classes[ind]),weights))
        incremental_dataset = incremental_dataset.shuffle(2048).batch(32)
        # load pre-trained keras model
        model = tf.keras.models.load_model(os.path.join(path_to_data, 'submodel_62_classes.hdf5'), compile=False)

        # use SGD optimizer
        optimizer = optimizers.Adam(lr=.0001)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # define callbacks 
        callbacks = [
            ModelCheckpoint(
                os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)), monitor='val_accuracy', 
                verbose=0, save_best_only=True, save_weights_only=False, mode='auto'),
            # you can use tensorboard --logdir path/to/log/dir to visualize the training parameters like loss and accuracy
            TensorBoard(os.path.join(logs, name,str(attempt))),
            #ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,patience=10, min_lr=0.000001, verbose=1)

        ] 

        # start training for 250 epochs
        model.fit_generator(
            incremental_dataset,
            epochs=500,
            callbacks=callbacks,
            validation_data=test_dataset)

        model.load_weights(os.path.join(logs, '{}_model_62_classes.hdf5'.format(name)))
        
        y_pred = model.predict(test_dataset)
        cm_b = confusion_matrix(test_classes, np.array(y_pred).argmax(axis=1))
        
        np.save(os.path.join('cm_probablity_'+name+'_'+str(attempt)),cm_b)
        pred_accu = accuracy_score(test_classes, np.array(y_pred).argmax(axis=1) )
        print(pred_accu)
        
        

Train for 201 steps, validate for 969 steps
Epoch 1/500
201/201 [==============================] - 4s 18ms/step - loss: 1.1064 - accuracy: 0.4358 - val_loss: 3.8547 - val_accuracy: 0.5711
Epoch 2/500
201/201 [==============================] - 3s 16ms/step - loss: 1.0766 - accuracy: 0.4594 - val_loss: 3.7898 - val_accuracy: 0.5237
Epoch 3/500
201/201 [==============================] - 3s 16ms/step - loss: 1.0536 - accuracy: 0.4638 - val_loss: 3.7278 - val_accuracy: 0.4798
Epoch 4/500
201/201 [==============================] - 3s 16ms/step - loss: 1.0322 - accuracy: 0.4720 - val_loss: 3.6676 - val_accuracy: 0.4536
Epoch 5/500
201/201 [==============================] - 3s 14ms/step - loss: 1.0062 - accuracy: 0.4695 - val_loss: 3.6093 - val_accuracy: 0.4357
Epoch 6/500
201/201 [==============================] - 3s 16ms/step - loss: 0.9884 - accuracy: 0.4789 - val_loss: 3.5525 - val_accuracy: 0.4261
Epoch 7/500
201/201 [==============================] - 3s 16ms/step - loss: 0.9658 - accurac

201/201 [==============================] - 3s 16ms/step - loss: 0.4408 - accuracy: 0.7318 - val_loss: 1.7945 - val_accuracy: 0.6573
Epoch 58/500
201/201 [==============================] - 3s 16ms/step - loss: 0.4351 - accuracy: 0.7356 - val_loss: 1.7753 - val_accuracy: 0.6607
Epoch 59/500
201/201 [==============================] - 3s 16ms/step - loss: 0.4309 - accuracy: 0.7376 - val_loss: 1.7564 - val_accuracy: 0.6644
Epoch 60/500
201/201 [==============================] - 3s 16ms/step - loss: 0.4299 - accuracy: 0.7421 - val_loss: 1.7382 - val_accuracy: 0.6675
Epoch 61/500
201/201 [==============================] - 3s 17ms/step - loss: 0.4215 - accuracy: 0.7432 - val_loss: 1.7204 - val_accuracy: 0.6703
Epoch 62/500
201/201 [==============================] - 3s 16ms/step - loss: 0.4165 - accuracy: 0.7463 - val_loss: 1.7029 - val_accuracy: 0.6738
Epoch 63/500
201/201 [==============================] - 3s 16ms/step - loss: 0.4123 - accuracy: 0.7490 - val_loss: 1.6854 - val_accuracy: 0.677

201/201 [==============================] - 3s 16ms/step - loss: 0.2706 - accuracy: 0.8182 - val_loss: 1.1443 - val_accuracy: 0.7586
Epoch 114/500
201/201 [==============================] - 3s 16ms/step - loss: 0.2697 - accuracy: 0.8204 - val_loss: 1.1376 - val_accuracy: 0.7592
Epoch 115/500
201/201 [==============================] - 3s 16ms/step - loss: 0.2664 - accuracy: 0.8211 - val_loss: 1.1313 - val_accuracy: 0.7597
Epoch 116/500
201/201 [==============================] - 3s 16ms/step - loss: 0.2645 - accuracy: 0.8213 - val_loss: 1.1251 - val_accuracy: 0.7604
Epoch 117/500
201/201 [==============================] - 3s 16ms/step - loss: 0.2630 - accuracy: 0.8229 - val_loss: 1.1189 - val_accuracy: 0.7610
Epoch 118/500
201/201 [==============================] - 3s 16ms/step - loss: 0.2619 - accuracy: 0.8227 - val_loss: 1.1127 - val_accuracy: 0.7617
Epoch 119/500
201/201 [==============================] - 3s 16ms/step - loss: 0.2613 - accuracy: 0.8240 - val_loss: 1.1067 - val_accuracy:

201/201 [==============================] - 3s 16ms/step - loss: 0.2003 - accuracy: 0.8536 - val_loss: 0.9037 - val_accuracy: 0.7838
Epoch 170/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1994 - accuracy: 0.8538 - val_loss: 0.9009 - val_accuracy: 0.7842
Epoch 171/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1990 - accuracy: 0.8552 - val_loss: 0.8984 - val_accuracy: 0.7842
Epoch 172/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1970 - accuracy: 0.8540 - val_loss: 0.8959 - val_accuracy: 0.7845
Epoch 173/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1964 - accuracy: 0.8544 - val_loss: 0.8932 - val_accuracy: 0.7848
Epoch 174/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1964 - accuracy: 0.8547 - val_loss: 0.8906 - val_accuracy: 0.7852
Epoch 175/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1965 - accuracy: 0.8557 - val_loss: 0.8881 - val_accuracy:

201/201 [==============================] - 3s 16ms/step - loss: 0.1610 - accuracy: 0.8760 - val_loss: 0.7950 - val_accuracy: 0.7935
Epoch 226/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1608 - accuracy: 0.8761 - val_loss: 0.7937 - val_accuracy: 0.7936
Epoch 227/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1623 - accuracy: 0.8768 - val_loss: 0.7924 - val_accuracy: 0.7937
Epoch 228/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1596 - accuracy: 0.8764 - val_loss: 0.7910 - val_accuracy: 0.7938
Epoch 229/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1586 - accuracy: 0.8760 - val_loss: 0.7898 - val_accuracy: 0.7938
Epoch 230/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1589 - accuracy: 0.8771 - val_loss: 0.7885 - val_accuracy: 0.7943
Epoch 231/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1586 - accuracy: 0.8780 - val_loss: 0.7872 - val_accuracy:

...

201/201 [==============================] - 3s 16ms/step - loss: 0.1586 - accuracy: 0.8772 - val_loss: 0.7897 - val_accuracy: 0.7941
Epoch 230/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1586 - accuracy: 0.8777 - val_loss: 0.7883 - val_accuracy: 0.7945
Epoch 231/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1577 - accuracy: 0.8779 - val_loss: 0.7869 - val_accuracy: 0.7947
Epoch 232/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1575 - accuracy: 0.8775 - val_loss: 0.7856 - val_accuracy: 0.7949
Epoch 233/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1571 - accuracy: 0.8779 - val_loss: 0.7844 - val_accuracy: 0.7946
Epoch 234/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1560 - accuracy: 0.8786 - val_loss: 0.7830 - val_accuracy: 0.7951
Epoch 235/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1559 - accuracy: 0.8793 - val_loss: 0.7818 - val_accuracy:

201/201 [==============================] - 3s 16ms/step - loss: 0.1340 - accuracy: 0.8952 - val_loss: 0.7337 - val_accuracy: 0.7998
Epoch 286/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1338 - accuracy: 0.8946 - val_loss: 0.7329 - val_accuracy: 0.7999
Epoch 287/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1329 - accuracy: 0.8949 - val_loss: 0.7321 - val_accuracy: 0.8001
Epoch 288/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1322 - accuracy: 0.8952 - val_loss: 0.7314 - val_accuracy: 0.8004
Epoch 289/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1317 - accuracy: 0.8953 - val_loss: 0.7306 - val_accuracy: 0.8003
Epoch 290/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1319 - accuracy: 0.8964 - val_loss: 0.7300 - val_accuracy: 0.8004
Epoch 291/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1309 - accuracy: 0.8964 - val_loss: 0.7294 - val_accuracy:

201/201 [==============================] - 3s 16ms/step - loss: 0.1151 - accuracy: 0.9082 - val_loss: 0.7006 - val_accuracy: 0.8025
Epoch 342/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1143 - accuracy: 0.9085 - val_loss: 0.7002 - val_accuracy: 0.8025
Epoch 343/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1138 - accuracy: 0.9091 - val_loss: 0.6997 - val_accuracy: 0.8026
Epoch 344/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1135 - accuracy: 0.9097 - val_loss: 0.6993 - val_accuracy: 0.8025
Epoch 345/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1144 - accuracy: 0.9102 - val_loss: 0.6987 - val_accuracy: 0.8026
Epoch 346/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1134 - accuracy: 0.9099 - val_loss: 0.6983 - val_accuracy: 0.8027
Epoch 347/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1130 - accuracy: 0.9102 - val_loss: 0.6978 - val_accuracy:

201/201 [==============================] - 3s 16ms/step - loss: 0.0997 - accuracy: 0.9228 - val_loss: 0.6796 - val_accuracy: 0.8041
Epoch 398/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0993 - accuracy: 0.9227 - val_loss: 0.6794 - val_accuracy: 0.8040
Epoch 399/500
201/201 [==============================] - 3s 16ms/step - loss: 0.1008 - accuracy: 0.9222 - val_loss: 0.6790 - val_accuracy: 0.8040
Epoch 400/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0988 - accuracy: 0.9233 - val_loss: 0.6786 - val_accuracy: 0.8043
Epoch 401/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0989 - accuracy: 0.9238 - val_loss: 0.6784 - val_accuracy: 0.8042
Epoch 402/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0988 - accuracy: 0.9238 - val_loss: 0.6780 - val_accuracy: 0.8042
Epoch 403/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0986 - accuracy: 0.9244 - val_loss: 0.6777 - val_accuracy:

201/201 [==============================] - 3s 16ms/step - loss: 0.0875 - accuracy: 0.9339 - val_loss: 0.6658 - val_accuracy: 0.8055
Epoch 454/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0876 - accuracy: 0.9336 - val_loss: 0.6657 - val_accuracy: 0.8056
Epoch 455/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0873 - accuracy: 0.9339 - val_loss: 0.6655 - val_accuracy: 0.8054
Epoch 456/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0871 - accuracy: 0.9344 - val_loss: 0.6652 - val_accuracy: 0.8056
Epoch 457/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0866 - accuracy: 0.9342 - val_loss: 0.6650 - val_accuracy: 0.8056
Epoch 458/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0867 - accuracy: 0.9350 - val_loss: 0.6648 - val_accuracy: 0.8057
Epoch 459/500
201/201 [==============================] - 3s 16ms/step - loss: 0.0865 - accuracy: 0.9344 - val_loss: 0.6646 - val_accuracy:

In [14]:
y_pred_x = model.predict(test_dataset)
    
cm_x = confusion_matrix(test_classes, np.array(y_pred_x).argmax(axis=1))

In [15]:
pred_accu = accuracy_score(test_classes, np.array(y_pred_x).argmax(axis=1) )
print(pred_accu)

0.8170967741935484


In [16]:
np.save('cm_probablity_ac01_1',cm_x)